In [2]:
import os
import sys
import numpy as np
from libra_py import data_conv

In [5]:
data_conv.vasp_to_xyz('C3N4_1x1.vasp')

[24, 32] ['C', 'N']
A        14.2415132523         0.0000000000         0.0000000000

B         0.0002151559        12.3343009930         0.0000000000

C         0.0018975023         0.0028002808        14.9999996186

